# BASIC LINEAR MODEL: EDSA Apple Prices Challenge

#The data description and procedure of solving the defined  problem

The first thing we need to do is import the necessary libabraries and data

import relevent liabraries convert this cell into code

import your data here  please convert this cell into code


Above we separate the Id column from our test data and you'll see why we do this later on

Confirmatory Data Analysis
from this section we are going to check our data (data set)

First we'll take a look at he first few rows pof our training data

 now we see our data frame convert this cell into cell  
e,g  train.head()

In the dataframe above we can see that multiple columns have null values, but we can't tell from just this how many columns have null values down the line that we can't see now and we also don't know how many missing values those columns have. The Pandas library can help us address this with the info method. This function tells us what columns are in the dataframe, how many null values they have and what datatype they are.

on this cell we are going to view our data info # convert this cell into code
e.g train.info()

So we can now see which columns have missing values and how much data in that column is missing. Let's take it a step further, let's try and find out what percentage of the data in each collumn the missing values account for with the function we've created below. 

this details which columns are null and have missing values

on this section we are going to see the missing value. converting this cell into code.

a well written algorithm will help you 